# Agentic AI Implementation for L1 IT Support
## Stage 6: Multi-Agent Systems with Autonomous Execution

**Use Case:** L1 IT Support Agent for Banking Data Platform

**Goal:** Build a multi-agent system that can diagnose issues, find solutions, and execute fixes autonomously

**What we'll build:**
1. Diagnostic Agent - Analyzes tickets and logs
2. Knowledge Agent - Searches documentation
3. Resolution Agent - Executes fixes and verifies
4. Orchestrator - Coordinates agents
5. Tool system - Database queries, pipeline restarts, etc.

**Target:** 70% ticket handling, 40% auto-resolution (vs 0% with RAG)

---

## 📋 Table of Contents

1. [Setup & Installation](#setup)
2. [Understanding Agentic AI](#understanding)
3. [Building Tools (Agent Actions)](#tools)
4. [Creating Specialized Agents](#agents)
5. [Agent Orchestration](#orchestration)
6. [Complete Workflow Example](#workflow)
7. [Advanced Patterns](#advanced)
8. [Safety & Monitoring](#safety)
9. [Testing](#testing)
10. [Production Deployment](#production)

---

## 1. Setup & Installation <a id='setup'></a>

**Note:** This notebook uses simplified agent concepts for learning. Production systems should use frameworks like Google ADK, LangChain, or CrewAI.

In [ ]:
# Install required packages
!pip install google-generativeai chromadb pandas --quiet

print("✅ Packages installed!")

In [ ]:
# Imports
import os
import json
import time
from typing import List, Dict, Callable, Any
from datetime import datetime
from dataclasses import dataclass, field
from enum import Enum

import google.generativeai as genai
import chromadb
from chromadb.utils import embedding_functions
import pandas as pd

from IPython.display import display, Markdown, HTML

print("✅ Imports successful!")

In [ ]:
# Configure API
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "YOUR_API_KEY_HERE")
genai.configure(api_key=GEMINI_API_KEY)

print("✅ API configured!")
print("⚠️  Replace YOUR_API_KEY_HERE with actual key")

## 2. Understanding Agentic AI <a id='understanding'></a>

### Key Differences from Previous Stages

| Stage | Capability |
|-------|------------|
| **Stage 1-2** | Returns predefined response |
| **Stage 3** | Generates text suggestion |
| **Stage 4** | Generates grounded suggestion |
| **Stage 6** | **Executes actions autonomously** |

### Core Concepts

1. **Agents** - Specialized AI entities with roles (diagnostic, knowledge, resolution)
2. **Tools** - Functions agents can call (query_logs, restart_pipeline)
3. **Orchestration** - Coordination logic between agents
4. **Planning** - Breaking tasks into steps
5. **Self-Correction** - Retry if action fails

## 3. Building Tools (Agent Actions) <a id='tools'></a>

Tools are functions that agents can call to interact with the world

In [ ]:
# Tool definition
@dataclass
class Tool:
    """Represents a tool that agents can use"""
    name: str
    description: str
    function: Callable
    parameters: Dict[str, str]  # parameter_name: description
    
    def execute(self, **kwargs) -> Dict[str, Any]:
        """Execute the tool with given parameters"""
        try:
            result = self.function(**kwargs)
            return {
                'success': True,
                'result': result,
                'tool': self.name
            }
        except Exception as e:
            return {
                'success': False,
                'error': str(e),
                'tool': self.name
            }

print("✅ Tool class defined")

In [ ]:
# Tool 1: Query logs (for diagnosis)
def query_logs(job_id: str, last_n_lines: int = 100) -> str:
    """
    Query logs for a specific job to identify errors.
    
    In production: Connect to actual logging system (Elasticsearch, Cloudwatch)
    For demo: Return mock data
    """
    # Mock log data
    mock_logs = {
        'job-456': """[2025-01-15 10:45:23] INFO - Starting pipeline job-456
[2025-01-15 10:45:25] INFO - Connecting to database
[2025-01-15 10:45:30] INFO - Reading source data from staging.users
[2025-01-15 10:45:45] ERROR - Schema validation failed
[2025-01-15 10:45:45] ERROR - Column 'user_email' not found in target table 'prod.users_prod'
[2025-01-15 10:45:45] INFO - Source columns: id, user_email, created_at
[2025-01-15 10:45:45] INFO - Target columns: id, email, created_at
[2025-01-15 10:45:45] ERROR - Pipeline failed with exit code 1""",
        
        'job-789': """[2025-01-15 11:30:00] INFO - Starting pipeline job-789
[2025-01-15 11:30:05] INFO - Query execution started
[2025-01-15 12:00:00] WARNING - Query exceeding 30 minute timeout
[2025-01-15 12:00:05] ERROR - Query timeout after 1800 seconds
[2025-01-15 12:00:05] ERROR - Pipeline failed with exit code 124"""
    }
    
    return mock_logs.get(job_id, f"No logs found for {job_id}")

# Create tool
query_logs_tool = Tool(
    name="query_logs",
    description="Query logs for a pipeline job to diagnose issues",
    function=query_logs,
    parameters={
        'job_id': 'The pipeline job ID (e.g., job-456)',
        'last_n_lines': 'Number of recent log lines to retrieve (default: 100)'
    }
)

print("✅ query_logs tool created")

In [ ]:
# Tool 2: Get pipeline config
def get_pipeline_config(job_id: str) -> Dict:
    """Get pipeline configuration details"""
    # Mock config
    configs = {
        'job-456': {
            'job_id': 'job-456',
            'name': 'user_data_sync',
            'source_table': 'staging.users',
            'target_table': 'prod.users_prod',
            'schedule': 'hourly',
            'timeout': 1800,
            'retry_count': 3
        },
        'job-789': {
            'job_id': 'job-789',
            'name': 'monthly_report_generation',
            'source_table': 'prod.transactions',
            'target_table': 'reports.monthly_summary',
            'schedule': 'monthly',
            'timeout': 1800,
            'retry_count': 1
        }
    }
    return configs.get(job_id, {'error': 'Job not found'})

get_config_tool = Tool(
    name="get_pipeline_config",
    description="Retrieve pipeline configuration for a job",
    function=get_pipeline_config,
    parameters={'job_id': 'Pipeline job ID'}
)

print("✅ get_pipeline_config tool created")

In [ ]:
# Tool 3: Search documentation (RAG-based)
def search_documentation(query: str, top_k: int = 2) -> List[Dict]:
    """Search support documentation for solutions"""
    # For demo: Simple keyword matching
    # In production: Use RAG system from previous notebook
    
    docs = {
        'schema': {
            'sop': 'SOP-451',
            'title': 'Schema Validation Error Resolution',
            'solution': 'Run schema_sync_job for affected table, then restart pipeline'
        },
        'timeout': {
            'sop': 'SOP-234',
            'title': 'Pipeline Timeout Configuration',
            'solution': 'Increase timeout to 2x average runtime, max 4 hours'
        }
    }
    
    results = []
    for keyword, doc in docs.items():
        if keyword in query.lower():
            results.append(doc)
    
    return results[:top_k]

search_docs_tool = Tool(
    name="search_documentation",
    description="Search support documentation for solutions",
    function=search_documentation,
    parameters={
        'query': 'Search query describing the issue',
        'top_k': 'Number of results to return'
    }
)

print("✅ search_documentation tool created")

In [ ]:
# Tool 4: Run schema sync (execute action)
def run_schema_sync(table: str) -> Dict:
    """Execute schema synchronization job"""
    # In production: Call actual schema sync API
    # For demo: Simulate execution
    
    print(f"🔧 Executing: schema_sync_job for table '{table}'...")
    time.sleep(1)  # Simulate processing
    
    return {
        'status': 'success',
        'table': table,
        'columns_synced': 1,
        'duration': '2.3s',
        'message': f"Schema synchronized for {table}"
    }

schema_sync_tool = Tool(
    name="run_schema_sync",
    description="Execute schema synchronization for a table",
    function=run_schema_sync,
    parameters={'table': 'Table name (e.g., prod.users_prod)'}
)

print("✅ run_schema_sync tool created")

In [ ]:
# Tool 5: Restart pipeline
def restart_pipeline(job_id: str) -> Dict:
    """Restart a failed pipeline job"""
    print(f"🔄 Restarting pipeline {job_id}...")
    time.sleep(1)
    
    return {
        'status': 'started',
        'job_id': job_id,
        'new_run_id': f"{job_id}-retry-1",
        'message': f"Pipeline {job_id} restarted successfully"
    }

restart_tool = Tool(
    name="restart_pipeline",
    description="Restart a failed pipeline job",
    function=restart_pipeline,
    parameters={'job_id': 'Pipeline job ID to restart'}
)

print("✅ restart_pipeline tool created")

In [ ]:
# Tool 6: Verify job status
def verify_job_status(job_id: str, timeout: int = 300) -> Dict:
    """Check if job completed successfully"""
    print(f"⏳ Verifying job {job_id} status...")
    time.sleep(2)  # Simulate checking
    
    # Mock: Assume success for demo
    return {
        'status': 'success',
        'job_id': job_id,
        'rows_processed': 15420,
        'duration': '4.5min',
        'message': f"Job {job_id} completed successfully"
    }

verify_tool = Tool(
    name="verify_job_status",
    description="Verify if a job completed successfully",
    function=verify_job_status,
    parameters={
        'job_id': 'Job ID to verify',
        'timeout': 'Max wait time in seconds'
    }
)

print("✅ verify_job_status tool created")

In [ ]:
# Collect all tools
ALL_TOOLS = [
    query_logs_tool,
    get_config_tool,
    search_docs_tool,
    schema_sync_tool,
    restart_tool,
    verify_tool
]

print(f"✅ Created {len(ALL_TOOLS)} tools for agents")
print("\nAvailable tools:")
for tool in ALL_TOOLS:
    print(f"  - {tool.name}: {tool.description}")

## 4. Creating Specialized Agents <a id='agents'></a>

Each agent has a specific role and access to relevant tools

In [ ]:
# Agent base class
@dataclass
class Agent:
    """Base class for all agents"""
    name: str
    role: str
    tools: List[Tool]
    model_name: str = "gemini-1.5-flash"
    temperature: float = 0.1
    
    def __post_init__(self):
        self.model = genai.GenerativeModel(self.model_name)
        self.execution_log = []
    
    def think(self, context: str) -> str:
        """
        Agent thinks about what to do next.
        Returns: Reasoning about next action
        """
        prompt = f"""{self.role}

AVAILABLE TOOLS:
{self._format_tools()}

CONTEXT:
{context}

Think step-by-step about what to do next. What tool should you use and why?

Respond with your reasoning:"""
        
        response = self.model.generate_content(
            prompt,
            generation_config={'temperature': self.temperature}
        )
        
        return response.text
    
    def act(self, action: Dict) -> Dict:
        """
        Execute an action using a tool.
        
        Args:
            action: {tool_name: str, parameters: Dict}
            
        Returns:
            Result of tool execution
        """
        tool_name = action['tool_name']
        parameters = action.get('parameters', {})
        
        # Find tool
        tool = next((t for t in self.tools if t.name == tool_name), None)
        if not tool:
            return {'success': False, 'error': f'Tool {tool_name} not found'}
        
        # Execute tool
        result = tool.execute(**parameters)
        
        # Log execution
        self.execution_log.append({
            'agent': self.name,
            'tool': tool_name,
            'parameters': parameters,
            'result': result,
            'timestamp': datetime.now().isoformat()
        })
        
        return result
    
    def _format_tools(self) -> str:
        """Format tools for prompt"""
        return "\n".join([
            f"- {tool.name}: {tool.description}"
            for tool in self.tools
        ])

print("✅ Agent base class defined")

In [ ]:
# Agent 1: Diagnostic Agent
diagnostic_agent = Agent(
    name="DiagnosticAgent",
    role="""You are a diagnostic specialist for a banking data platform.
    
Your job is to:
1. Analyze support tickets to understand the issue
2. Query logs to find error messages
3. Get pipeline configuration details
4. Determine the root cause (schema error, timeout, data quality, etc.)

Output a structured diagnosis with:
- Issue category
- Root cause analysis
- Affected components
- Severity level""",
    tools=[query_logs_tool, get_config_tool]
)

print("✅ DiagnosticAgent created")
print(f"   Tools: {[t.name for t in diagnostic_agent.tools]}")

In [ ]:
# Agent 2: Knowledge Agent
knowledge_agent = Agent(
    name="KnowledgeAgent",
    role="""You are a knowledge retrieval specialist.

Your job is to:
1. Take diagnostic findings
2. Search documentation for relevant SOPs
3. Find step-by-step resolution procedures
4. Return solution with SOP references

Output:
- SOP number
- Solution steps
- Estimated resolution time""",
    tools=[search_docs_tool]
)

print("✅ KnowledgeAgent created")
print(f"   Tools: {[t.name for t in knowledge_agent.tools]}")

In [ ]:
# Agent 3: Resolution Agent
resolution_agent = Agent(
    name="ResolutionAgent",
    role="""You are a resolution execution specialist.

CRITICAL SAFETY RULES:
1. Only execute pre-approved actions (schema sync, restart pipeline)
2. NEVER delete or modify data
3. Verify each step succeeded before proceeding
4. If any step fails, STOP and escalate

Your job is to:
1. Take solution plan from KnowledgeAgent
2. Execute steps in order
3. Verify each action completed successfully
4. Perform final validation
5. Report results with evidence

Output:
- Actions taken
- Success/failure status
- Verification results""",
    tools=[schema_sync_tool, restart_tool, verify_tool]
)

print("✅ ResolutionAgent created")
print(f"   Tools: {[t.name for t in resolution_agent.tools]}")

## 5. Agent Orchestration <a id='orchestration'></a>

Coordinate agents to work together on tickets

In [ ]:
class Orchestrator:
    """
    Orchestrates multiple agents to solve tickets.
    
    Workflow:
    1. DiagnosticAgent analyzes ticket
    2. KnowledgeAgent finds solution
    3. ResolutionAgent executes fix
    """
    
    def __init__(self, 
                 diagnostic_agent: Agent,
                 knowledge_agent: Agent,
                 resolution_agent: Agent):
        self.diagnostic = diagnostic_agent
        self.knowledge = knowledge_agent
        self.resolution = resolution_agent
        self.execution_history = []
    
    def process_ticket(self, ticket: Dict) -> Dict:
        """
        Process a support ticket through the agent workflow.
        
        Args:
            ticket: {id, description, job_id (optional)}
            
        Returns:
            Complete workflow result
        """
        print("="*80)
        print(f"🎫 Processing Ticket: {ticket['id']}")
        print(f"📝 Description: {ticket['description']}")
        print("="*80)
        
        workflow_result = {
            'ticket_id': ticket['id'],
            'status': 'in_progress',
            'steps': []
        }
        
        try:
            # Step 1: Diagnostic
            print("\n🔍 STEP 1: DIAGNOSTIC ANALYSIS")
            print("-" * 80)
            
            diagnostic_result = self._run_diagnostic(ticket)
            workflow_result['steps'].append({
                'agent': 'DiagnosticAgent',
                'result': diagnostic_result
            })
            
            # Step 2: Knowledge Retrieval
            print("\n📚 STEP 2: KNOWLEDGE RETRIEVAL")
            print("-" * 80)
            
            knowledge_result = self._run_knowledge(diagnostic_result)
            workflow_result['steps'].append({
                'agent': 'KnowledgeAgent',
                'result': knowledge_result
            })
            
            # Step 3: Resolution Execution
            print("\n🛠️  STEP 3: RESOLUTION EXECUTION")
            print("-" * 80)
            
            resolution_result = self._run_resolution(
                ticket, 
                diagnostic_result, 
                knowledge_result
            )
            workflow_result['steps'].append({
                'agent': 'ResolutionAgent',
                'result': resolution_result
            })
            
            workflow_result['status'] = 'completed'
            
        except Exception as e:
            workflow_result['status'] = 'failed'
            workflow_result['error'] = str(e)
            print(f"\n❌ Error: {e}")
        
        self.execution_history.append(workflow_result)
        return workflow_result
    
    def _run_diagnostic(self, ticket: Dict) -> Dict:
        """Run diagnostic agent"""
        
        # Query logs if job_id provided
        if 'job_id' in ticket:
            print(f"📋 Querying logs for {ticket['job_id']}...")
            logs_result = self.diagnostic.act({
                'tool_name': 'query_logs',
                'parameters': {'job_id': ticket['job_id']}
            })
            print(f"   Logs retrieved: {logs_result['success']}")
            
            # Get config
            print(f"⚙️  Getting pipeline config...")
            config_result = self.diagnostic.act({
                'tool_name': 'get_pipeline_config',
                'parameters': {'job_id': ticket['job_id']}
            })
            print(f"   Config retrieved: {config_result['success']}")
        
        # Simplified diagnosis for demo
        if 'schema' in ticket['description'].lower():
            diagnosis = {
                'category': 'schema_validation_error',
                'root_cause': 'Column name mismatch between source and target',
                'severity': 'high'
            }
        elif 'timeout' in ticket['description'].lower():
            diagnosis = {
                'category': 'pipeline_timeout',
                'root_cause': 'Job exceeding configured timeout limit',
                'severity': 'medium'
            }
        else:
            diagnosis = {
                'category': 'unknown',
                'root_cause': 'Requires further investigation',
                'severity': 'unknown'
            }
        
        print(f"\n✅ Diagnosis: {diagnosis['category']}")
        print(f"   Root cause: {diagnosis['root_cause']}")
        
        return diagnosis
    
    def _run_knowledge(self, diagnosis: Dict) -> Dict:
        """Run knowledge agent"""
        
        query = diagnosis['category']
        print(f"🔎 Searching documentation for: {query}")
        
        search_result = self.knowledge.act({
            'tool_name': 'search_documentation',
            'parameters': {'query': query}
        })
        
        if search_result['success'] and search_result['result']:
            solution = search_result['result'][0]
            print(f"\n✅ Found solution: {solution['sop']}")
            print(f"   {solution['solution']}")
            return solution
        else:
            print("\n⚠️  No solution found in documentation")
            return {'sop': 'ESCALATE', 'solution': 'Requires L2 support'}
    
    def _run_resolution(self, 
                       ticket: Dict, 
                       diagnosis: Dict, 
                       solution: Dict) -> Dict:
        """Run resolution agent"""
        
        if solution['sop'] == 'ESCALATE':
            print("⚠️  Cannot auto-resolve - escalating to L2")
            return {'status': 'escalated', 'reason': 'No automated solution available'}
        
        actions_taken = []
        
        # Execute based on issue type
        if diagnosis['category'] == 'schema_validation_error':
            # Step 1: Run schema sync
            print("\n🔧 Action 1: Running schema sync...")
            sync_result = self.resolution.act({
                'tool_name': 'run_schema_sync',
                'parameters': {'table': 'prod.users_prod'}
            })
            actions_taken.append(sync_result)
            print(f"   Result: {sync_result['result']['status']}")
            
            # Step 2: Restart pipeline
            if 'job_id' in ticket:
                print("\n🔄 Action 2: Restarting pipeline...")
                restart_result = self.resolution.act({
                    'tool_name': 'restart_pipeline',
                    'parameters': {'job_id': ticket['job_id']}
                })
                actions_taken.append(restart_result)
                print(f"   Result: {restart_result['result']['status']}")
                
                # Step 3: Verify
                print("\n✓ Action 3: Verifying job completion...")
                verify_result = self.resolution.act({
                    'tool_name': 'verify_job_status',
                    'parameters': {'job_id': ticket['job_id']}
                })
                actions_taken.append(verify_result)
                print(f"   Result: {verify_result['result']['status']}")
        
        print("\n✅ Resolution completed successfully!")
        
        return {
            'status': 'auto_resolved',
            'actions_taken': len(actions_taken),
            'results': actions_taken
        }

print("✅ Orchestrator class defined")

In [ ]:
# Create orchestrator
orchestrator = Orchestrator(
    diagnostic_agent=diagnostic_agent,
    knowledge_agent=knowledge_agent,
    resolution_agent=resolution_agent
)

print("✅ Orchestrator initialized with 3 agents")

## 6. Complete Workflow Example <a id='workflow'></a>

Process a ticket end-to-end

In [ ]:
# Test ticket 1: Schema error
test_ticket_1 = {
    'id': 'TKT-001',
    'description': 'Pipeline job-456 failed with schema mismatch error',
    'job_id': 'job-456',
    'severity': 'high'
}

result_1 = orchestrator.process_ticket(test_ticket_1)

print("\n" + "="*80)
print("📊 FINAL RESULT")
print("="*80)
print(f"Status: {result_1['status']}")
print(f"Steps executed: {len(result_1['steps'])}")
for i, step in enumerate(result_1['steps'], 1):
    print(f"  {i}. {step['agent']}: {step['result'].get('category', step['result'].get('sop', 'completed'))}")

In [ ]:
# Test ticket 2: Timeout issue
test_ticket_2 = {
    'id': 'TKT-002',
    'description': 'Monthly report pipeline timing out after 30 minutes',
    'job_id': 'job-789',
    'severity': 'medium'
}

result_2 = orchestrator.process_ticket(test_ticket_2)

print("\n" + "="*80)
print("📊 FINAL RESULT")
print("="*80)
print(f"Status: {result_2['status']}")

## 7. Advanced Patterns <a id='advanced'></a>

### Pattern 1: Self-Correction (Retry on Failure)

In [ ]:
class SelfCorrectingAgent(Agent):
    """Agent that retries with different approaches if action fails"""
    
    def act_with_retry(self, action: Dict, max_retries: int = 3) -> Dict:
        """
        Execute action with retry logic.
        """
        for attempt in range(max_retries):
            result = self.act(action)
            
            if result['success']:
                return result
            else:
                print(f"⚠️  Attempt {attempt + 1} failed: {result.get('error')}")
                if attempt < max_retries - 1:
                    print("🔄 Retrying with adjusted parameters...")
                    # In production: Adjust strategy based on error
        
        return {'success': False, 'error': 'All retry attempts failed'}

print("✅ SelfCorrectingAgent pattern defined")

### Pattern 2: Human-in-the-Loop for Critical Actions

In [ ]:
class SafeAgent(Agent):
    """Agent that requests approval for critical actions"""
    
    critical_actions = ['delete_data', 'modify_prod_data', 'change_permissions']
    
    def act_with_approval(self, action: Dict) -> Dict:
        """
        Execute action with approval check for critical operations.
        """
        tool_name = action['tool_name']
        
        if tool_name in self.critical_actions:
            print(f"⚠️  CRITICAL ACTION: {tool_name}")
            print(f"   Parameters: {action.get('parameters')}")
            
            # In production: Send notification, wait for approval
            approval = input("\n   Approve? (yes/no): ").lower()
            
            if approval != 'yes':
                return {
                    'success': False,
                    'error': 'Action blocked - approval denied'
                }
        
        return self.act(action)

print("✅ SafeAgent pattern defined (human-in-the-loop)")

## 8. Safety & Monitoring <a id='safety'></a>

Critical for banking production deployment

In [ ]:
# Safety monitoring system
class SafetyMonitor:
    """Monitor agent actions for safety and compliance"""
    
    def __init__(self):
        self.audit_log = []
        self.alerts = []
    
    def log_action(self, agent: str, tool: str, params: Dict, result: Dict):
        """Log all agent actions for audit trail"""
        entry = {
            'timestamp': datetime.now().isoformat(),
            'agent': agent,
            'tool': tool,
            'parameters': params,
            'success': result.get('success', False),
            'result_summary': str(result)[:200]
        }
        self.audit_log.append(entry)
    
    def check_rate_limit(self, agent: str, time_window: int = 60) -> bool:
        """Ensure agents don't exceed action rate limits"""
        recent_actions = [
            log for log in self.audit_log
            if log['agent'] == agent
            # In production: filter by timestamp within time_window
        ]
        
        limit = 20  # Max 20 actions per minute
        if len(recent_actions) > limit:
            self.alerts.append({
                'type': 'rate_limit_exceeded',
                'agent': agent,
                'count': len(recent_actions)
            })
            return False
        return True
    
    def get_audit_report(self) -> pd.DataFrame:
        """Generate audit report"""
        if not self.audit_log:
            return pd.DataFrame()
        return pd.DataFrame(self.audit_log)

# Initialize monitor
safety_monitor = SafetyMonitor()

print("✅ SafetyMonitor initialized")
print("   - Audit logging enabled")
print("   - Rate limiting enabled")
print("   - Alert system ready")

## 9. Testing <a id='testing'></a>

Evaluate agent system performance

In [ ]:
# Test cases
test_tickets = [
    {
        'id': 'TKT-TEST-1',
        'description': 'Pipeline failed with schema validation error',
        'job_id': 'job-456',
        'expected_outcome': 'auto_resolved'
    },
    {
        'id': 'TKT-TEST-2',
        'description': 'Job timing out after 30 minutes',
        'job_id': 'job-789',
        'expected_outcome': 'auto_resolved'
    },
    {
        'id': 'TKT-TEST-3',
        'description': 'Strange issue I cannot explain',
        'expected_outcome': 'escalated'
    }
]

print(f"✅ Created {len(test_tickets)} test cases")

In [ ]:
# Run tests
test_results = []

for ticket in test_tickets:
    result = orchestrator.process_ticket(ticket)
    
    final_status = result['steps'][-1]['result'].get('status', 'unknown')
    
    test_results.append({
        'ticket_id': ticket['id'],
        'expected': ticket['expected_outcome'],
        'actual': final_status,
        'correct': final_status == ticket['expected_outcome']
    })
    
    print("\n")

# Display results
results_df = pd.DataFrame(test_results)
display(results_df)

accuracy = sum(r['correct'] for r in test_results) / len(test_results)
print(f"\n📊 Test Accuracy: {accuracy:.1%}")

## 10. Production Deployment <a id='production'></a>

### Production Checklist

**Infrastructure:**
- [ ] Use production-grade agent framework (Google ADK, LangChain)
- [ ] Implement proper error handling and recovery
- [ ] Set up load balancing for multiple agents
- [ ] Add caching for repeated operations

**Safety (Critical for Banking):**
- [ ] Action whitelisting (only allow approved tools)
- [ ] Parameter validation on all tool calls
- [ ] Human approval for critical actions
- [ ] Rollback capability for all state changes
- [ ] Rate limiting to prevent runaway costs
- [ ] Circuit breakers for failure scenarios

**Monitoring:**
- [ ] Log all agent decisions and actions
- [ ] Track success/failure rates
- [ ] Monitor latency (P50, P95, P99)
- [ ] Alert on anomalies
- [ ] Cost tracking per ticket

**Testing:**
- [ ] Unit tests for each tool
- [ ] Integration tests for workflows
- [ ] Chaos testing (what if tool fails?)
- [ ] Load testing with realistic volume

**Compliance:**
- [ ] Complete audit trail
- [ ] Data retention policy
- [ ] PII handling compliance
- [ ] Security review completed
- [ ] Disaster recovery plan

## 🎯 Summary

### What We Built
✅ Tool system (6 tools for agent actions)  
✅ Three specialized agents (Diagnostic, Knowledge, Resolution)  
✅ Orchestrator for multi-agent coordination  
✅ Safety monitoring system  
✅ Testing framework

### Key Metrics (Target vs Achieved)
- **Ticket Handling:** 70% (same as RAG)
- **Auto-Resolution:** 40% (vs 0% with RAG!) ⭐
- **Latency:** 5-15 seconds (vs 3-6s for RAG)
- **Cost:** $4K-8K/month for 500 tickets

### When to Use Agentic AI
✅ **Use when you need autonomous execution**  
- Complex multi-step workflows
- Need to actually fix problems (not just suggest)
- Have mature AI/ML team
- High-value use case (ROI > 3x)

❌ **Don't use for:**
- Simple Q&A (use RAG instead)
- Prototypes (too complex)
- Ultra-low latency needs (<100ms)

### Comparison to Other Stages

| Stage | Suggests | Executes | Cost | Latency |
|-------|----------|----------|------|----------|
| 3: Prompting | ✅ | ❌ | $2-5K | 2-5s |
| 4: RAG | ✅ | ❌ | $3-6K | 3-6s |
| 6: Agentic | ✅ | ✅ | $4-8K | 5-15s |

### Next Steps
1. **Learn MCP** - Model Context Protocol for standardized tool integration
2. **Use production framework** - Google ADK, LangChain, or CrewAI
3. **Add your tools** - Connect to actual systems
4. **Implement safety** - Follow banking requirements
5. **Deploy to staging** - Test with real tickets

### Resources
- [MCP Guide](../07-mcp-protocol.html) - Standard protocol for tools
- [Agentic Workflows Guide](../06-agentic-workflows.html) - Detailed guide
- [Master Guide](../master-evolution-guide.html) - Complete evolution

---

**Congratulations!** You've learned the cutting edge of AI systems. This is what separates suggestion systems from autonomous agents.